In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

!pip install -q plotnine
from plotnine import *

import tensorflow as tf
from tensorflow import keras

import math
import keras

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, ElasticNetCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.svm import SVC

from datetime import datetime
import urllib

## Get wheather data from URL

In [16]:
def get_weather_data(start_year, start_month, start_day, end_year, end_month, end_day):
    url = f'https://meteo.physic.ut.ee/et/archive.php?do=data&begin%5Byear%5D={start_year}&begin%5Bmon%5D={start_month}&begin%5Bmday%5D={start_day}&end%5Byear%5D={end_year}&end%5Bmon%5D={end_month}&end%5Bmday%5D={end_day}&9=1&12=1&10=1&15=1&16=1&14=1&ok=+Esita+p%C3%A4ring+'
    file = urllib. request. urlopen(url)
    lines = []
    for line in file:
        decoded_line = line.decode("utf-8")
        new_line = decoded_line.replace(" ","").strip()
        lines.append(new_line)
    with open('data.csv', 'w') as f:
        f.write('\n'.join(lines))
    data = pd.read_csv('data.csv')
    data.columns = ['timestamp', 'temperature', 'humidity', 'atm_pressure', 'windspeed', 'wind_direction', 'precipitation', 'radiation_flux']
    return data

In [17]:
data = get_weather_data(2020,12,1,2021,11,30)
data.sample(10)

,timestamp,temperature,humidity,atm_pressure,windspeed,wind_direction,precipitation,radiation_flux
42079,2021-04-1717:50:00,16.445986,17.797370,1023.707033,5.460511,51.656338,0.0,205.281128
85191,2021-09-1410:30:00,10.763913,65.476061,1011.311233,7.096696,297.536053,0.0,273.136588
87661,2021-09-2300:20:00,5.063063,92.365722,1016.263833,1.859242,0.645392,0.0,0.000000
78302,2021-08-2112:25:00,17.940612,59.929043,1011.007267,3.515541,253.019791,0.0,721.902367
73724,2021-08-0514:55:00,19.543360,43.978793,1014.799600,3.318548,83.974961,0.0,178.289834
17721,2021-01-2303:00:00,1.766247,88.994746,996.132300,3.679681,202.635044,0.0,4.866066
48754,2021-05-1022:05:00,18.282245,44.212924,1018.874200,1.533341,188.610508,0.0,0.000000
54622,2021-05-3107:05:00,9.758715,60.190126,1022.987433,0.662936,170.511629,0.0,286.175298
89773,2021-09-3008:20:00,5.028936,83.054158,1030.890767,4.115406,138.559775,0.0,37.884222
20349,2021-02-0106:00:00,-4.650378,91.548911,996.650367,4.333895,221.339705,0.0,0.913042


In [18]:
def break_up_time(data):
    data["year"]= data.apply(lambda row: row["timestamp"][0:4], axis=1)
    data["month"] = data.apply(lambda row: row["timestamp"][5:7], axis=1)
    data["day"] = data.apply(lambda row: row["timestamp"][8:10], axis=1)
    data["hour"] = data.apply(lambda row: row["timestamp"][10:12], axis=1)
    data["minute"] = data.apply(lambda row: row["timestamp"][13:15], axis=1)
    data.year = data.year.astype('int')
    data.month = data.month.astype('int')
    data.day = data.day.astype('int')
    data.hour = data.hour.astype('int')
    data.minute = data.minute.astype('int')
    data["timestamp"] = data.apply(lambda row: datetime.strptime(row["timestamp"],"%Y-%m-%d%H:%M:%S"), axis=1)
    return data

def fill_and_correct(data):
    # if there is no rain and no snow:
    #data['snow'] = data['snow'].fillna(0)
    data['precipitation'] = data['precipitation'].fillna(0)
    # for other 
    data['windspeed'] = data['windspeed'].fillna(method='backfill')
    data['wind_direction'] = data['wind_direction'].fillna(method='backfill')
    data['temperature'] = data['temperature'].fillna(method='backfill')
    data['humidity'] = data['humidity'].fillna(method='backfill')
    data['atm_pressure'] = data['atm_pressure'].fillna(method='backfill')
    data['radiation_flux'] = data['radiation_flux'].fillna(method='backfill')
    data['radiation_in_future'] = data['radiation_flux'].shift(-24)
    data = data.dropna()
    return data

def cleansing(data):
    data = break_up_time(data)
    data = fill_and_correct(data)
    return data

In [19]:
data = cleansing(data)
data.sample(5)

,timestamp,temperature,humidity,atm_pressure,windspeed,wind_direction,precipitation,radiation_flux,year,month,day,hour,minute,radiation_in_future
24967,2021-02-17 06:50:00,-10.716550,81.590764,1019.218433,3.472478,71.180718,0.0,4.840634,2021,2,17,6,50,60.110891
1455,2020-12-06 01:15:00,2.231117,87.601286,1020.911667,4.434881,196.908714,0.0,0.277078,2020,12,6,1,15,0.338554
55769,2021-06-04 06:40:00,12.711627,80.669081,1026.744500,1.311531,293.208294,0.0,87.029119,2021,6,4,6,40,247.547243
77512,2021-08-18 18:35:00,16.035082,81.053379,1000.938967,6.143024,203.238421,0.0,66.733965,2021,8,18,18,35,2.890433
20597,2021-02-02 02:40:00,-4.713614,86.053126,1007.413667,3.444037,222.765528,0.0,0.783095,2021,2,2,2,40,4.853537


In [20]:
data.dtypes

timestamp              datetime64[ns]
temperature                   float64
humidity                      float64
atm_pressure                  float64
windspeed                     float64
wind_direction                float64
precipitation                 float64
radiation_flux                float64
year                            int64
month                           int64
day                             int64
hour                            int64
minute                          int64
radiation_in_future           float64
dtype: object

In [21]:
data.isnull().sum()

timestamp              0
temperature            0
humidity               0
atm_pressure           0
windspeed              0
wind_direction         0
precipitation          0
radiation_flux         0
year                   0
month                  0
day                    0
hour                   0
minute                 0
radiation_in_future    0
dtype: int64

plt.figure(figsize=(8, 12))

heatmap = sns.heatmap(corr_table, vmin=-1, vmax=1, annot=True, cmap='BrBG')

heatmap.set_title('Solar Irradiation Correlation, Tartu (2013-2021)', fontdict={'fontsize':18}, pad=25)
plt.tight_layout()
plt.savefig('corr.png', dpi=300)

# Training, validation, predicting

Split the data into test and training data:

In [ ]:
X = data[['year', 'month', 'day', 'hour', 'minute', 'temperature',
         'humidity', 'atmospheric_pressure', 'wind_speed', 'wind_direction',
         'precipitation', 'radiation_flux']] # 'snow'
y = data['rad_flux_infuture']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
print('X ',X.shape,', X_train ',X_train.shape,', X_test ',X_test.shape)
print('y ',y.shape,', y_train ',y_train.shape,', y_test ',y_test.shape) 

In [ ]:

RFR = RandomForestRegressor(n_estimators=20,random_state=0)
RFR.fit(X_train,y_train)


In [ ]:
RFR.score(X_test,y_test)

In [ ]:
y_pred = RFR.predict(X_test)

In [ ]:
plt.figure(figsize=(16,9))
plt.plot(np.arange(X_test.shape[0]),y_test)
plt.plot(np.arange(X_test.shape[0]),y_pred,'-',alpha=0.6)
plt.show()


In [ ]:
training_data = data[data.year<2021]
test_data = data[data.year==2021]
X_train = training_data[['year', 'month', 'day', 'hour', 'minute', 'temperature',
         'humidity', 'atmospheric_pressure', 'wind_speed', 'wind_direction',
         'precipitation', 'radiation_flux']]
y_train = training_data['rad_flux_infuture']
X_test = test_data[['year', 'month', 'day', 'hour', 'minute', 'temperature',
         'humidity', 'atmospheric_pressure', 'wind_speed', 'wind_direction',
         'precipitation', 'radiation_flux']]
y_test = test_data['rad_flux_infuture']

In [ ]:
RFR = RandomForestRegressor(n_estimators=20,random_state=0)
RFR.fit(X_train,y_train)

In [ ]:
RFR.score(X_test,y_test)

In [ ]:
y_pred=RFR.predict(X_test)

In [ ]:
plt.figure(figsize=(16,9))
plt.plot(np.arange(X_test.shape[0]),y_test)
plt.plot(np.arange(X_test.shape[0]),y_pred,'-',alpha=0.6)
plt.show()


In [ ]:
ERR = ExtraTreesRegressor()
ERR.fit(X_train,y_train)

In [ ]:
ERR.score(X_test,y_test)

In [ ]:
y_pred = ERR.predict(X_test)

In [ ]:
plt.figure(figsize=(16,9))
plt.plot(np.arange(X_test.shape[0]),y_test,label="True")
plt.plot(np.arange(X_test.shape[0]),y_pred,'-',alpha=0.6,label="Prediction")
plt.legend()
plt.title("2021")
plt.show()

In [ ]:
LinR = LinearRegression()
LinR.fit(X_train,y_train)
LinR.score(X_test,y_test)

In [ ]:
LR = Ridge()
LR.fit(X_train,y_train)
LR.score(X_test,y_test)

In [ ]:
LL = Lasso()
LL.fit(X_train,y_train)
LL.score(X_test,y_test)

In [ ]:
E = ElasticNet()
E.fit(X_train,y_train)
E.score(X_test,y_test)

# PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca=PCA(n_components=0.9)
pca.fit(X_train)
X_train_compressed = pca.transform(X_train)
X_test_compressed = pca.transform(X_test)

In [ ]:
X_train_compressed.shape

In [ ]:
RF = RandomForestRegressor()
RF.fit(X_train_compressed,y_train)
RF.score(X_test_compressed,y_test)

Worse!!!

# XGBoost

In [ ]:
!pip install xgboost

In [ ]:
import xgboost as xgb


# XGBoosts wants data to be wrapped into special formats
dtrain = xgb.DMatrix(X_train,y_train)
dtest = xgb.DMatrix(X_test,y_test)

# most meaningful parameters
param_list = [("objective", "multi:softmax"), ("eval_metric", "merror"), ("num_class", 10)]

# Number of trees
n_rounds = 600

# if nothing seems to improve for 50 iterations - stop
early_stopping = 50

# train for training and test for ... validation!    
eval_list = [(dtrain, "train"), (dtest, "validation")]

# 1,2,3.. go!
bst = xgb.train(param_list, dtrain, n_rounds, evals=eval_list, early_stopping_rounds=early_stopping, verbose_eval=True)


In [ ]:
ypred = bst.predict(dtest) #, iteration_range=(0, bst.best_iteration + 1)) # this iteration_range do not work, but was in the documentation page.. weird
ypred.shape
print(f'Accuracy of XGBoost is {(np.sum(ypred==test_labels)/test_labels.shape[0])*100}%')